In [1]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt'
)
text = open(path).read().lower()
print('Corpus length:', len(text))

/home/n/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


614400/600901 [==============================] - 2s 3us/step
('Corpus length:', 600901)


In [7]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen]) 
    
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Number of unique characters:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorisation')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

('Number of sequences:', 200281)
('Number of unique characters:', 59)
Vectorisation


In [10]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
          
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               96256     
_________________________________________________________________
dense_1 (Dense)              (None, 59)                7611      
Total params: 103,867
Trainable params: 103,867
Non-trainable params: 0
_________________________________________________________________


In [11]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


Define a function that reweights the original probability of next character, to allow us to change the stochastic temperature to make things more/less predictable from original text.

In [13]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [14]:
import random
import sys

for epoch in range(1, 60):
    print('Epoch:', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
       

('Epoch:', 1)
Epoch 1/1
200281/200281 [==============================] - 90s 452us/step - loss: 1.6303
--- Generating with seed: "haken, torn apart, cast forth--it
comprehends not itself wha"
('----- temperature:', 0.2)
haken, torn apart, cast forth--it
comprehends not itself what is the stands the even and all the state and a conscience. the would even a conscience. the same to the is the ages and a contradible of the conscience. the such a contradite and sense of the exception of the same to the state of the exception of the stands to the sense of the exception of the conscience. the same in the same to be one is the believed in the same the stands the state of the stan('----- temperature:', 0.5)
is the believed in the same the stands the state of the standabole for the have should be opinion as it is no believed by the one's present have the one is german of the bask, and a become not in the fact all to it is this possible. the exceptions, be so much or the have in generally in from

:origing noting of really sone from ithest idea of thousabli. here as
leptinably of and
unwerfenchies which of the grostate know often eneatese who "thor
philosophy--the going would kanimed? thereby
to lookid to type its
ow preesal, fee wan
each origin cogous and advinal, and
puw ancoom and
grest,
object of
the ciusurg a lo"g--i lent of hardness" it of all one auchologitifs and rovely erse
that herelyhsg, toould that broughtes ('Epoch:', 5)
Epoch 1/1
200281/200281 [==============================] - 89s 445us/step - loss: 1.4478
--- Generating with seed: "aming--for he is as
restless and aimless in his course as if"
('----- temperature:', 0.2)
aming--for he is as
restless and aimless in his course as if the sense of the fact the the such the superioricy as the such the such a sure to himself the such a such a person as the more and the sense of the fearst the suspect the most designation of the fundamental the such a such the such the more as the words of the spirit the condition of the

for which oun means of a naturald here with it is conscious verist anyageists of all say to extemn of
mesticitation, are the intellection
in have to so, suff better. an also the proxical alsos. the would certainty of most fenthant oner than much perfict facts. to on those sphilongmos; above tofer of a
end is silence is abjecism? it is
exception.ib('----- temperature:', 1.2)
ve tofer of a
end is silence is abjecism? it is
exception.ib" in trum weavma.. hower but the experary? to to after restates, nemise is
gromony iss! eccardence of
the doubler.

1tem what senrationnment, soul of trmar. that whenge who sounddersel cannments of
a wrusponging and oftenable bound--through coven, menar appa itual
dissol
and involid lottermen as
the truth regreead should being duscaor, the thought inflict. wherher
will at muciany--for it goned tha('Epoch:', 9)
Epoch 1/1
200281/200281 [==============================] - 87s 434us/step - loss: 1.3960
--- Generating with seed: "and goodness in the
world to thro

KeyboardInterrupt: 